In [52]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
import os
import json
import dotenv

dotenv.load_dotenv()

True

In [54]:
from typing import TypedDict, Annotated, List, Dict, Literal
from typing_extensions import TypedDict
from IPython.display import display, Markdown, Image
import operator

In [55]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

In [56]:
from langgraph.graph import START, END, StateGraph
from langgraph.types import Command, Send
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage
from langchain_experimental.utilities import PythonREPL
from langgraph.graph.message import add_messages

In [57]:
from src.utils_trans import search_flight_details

In [58]:
def get_system_prompt_tool(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " Always format your final answer as a JSON string."
        f"\n{suffix}"
    )

In [59]:
def get_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Always format your final answer as a JSON string."
        f"\n{suffix}"
    )

In [60]:
llm = ChatDeepSeek(model="deepseek-chat",
                   api_key=os.getenv("DEEPSEEK_API_KEY"))

In [61]:
class State(TypedDict):
    messages: Annotated[List, add_messages]
    trip_start_date: str
    trip_end_date: str
    traveler_preferences: Dict
    start_flight: str
    return_flight: str
    

    flight_info: Dict
    total_schedule: Dict 
    report_markdown: str

In [62]:
@tool
def get_flight_data(start_flight: str, return_flight: str) -> str:
    """
    Retrieve flight details for given flight numbers.
    """
    result = search_flight_details(start_flight, return_flight)
    json.dumps(result)
    return json.dumps(result)

In [63]:
flight_data_agent = create_react_agent(
    llm,
    tools=[get_flight_data],
    prompt=get_system_prompt_tool(
        "You are a travel planning assistant specializing in retrieving round-trip flight information. "
        "Use the get_flight_data tool with provided flight numbers and return its output directly as a JSON string. "
        "Do NOT include any explanatory text, comments, or non-JSON content. "
        "Return an empty JSON object {} if no flight data is available or if the tool fails."
    ),
)

### Create Graph


In [64]:
def flight_data_node(state: State) -> State:
    start_flight = state['start_flight']
    return_flight = state['return_flight']
    
    # Run agent
    agent_input = {"messages": state['messages'] + [HumanMessage(content=f"Get data for {start_flight} and {return_flight}")]}
    agent_output = flight_data_agent.invoke(agent_input)

    # Parse safely

    final_content = agent_output['messages'][-1].content
    flight_info = json.loads(final_content)

    
    # Update state with extracted dates (adjust keys based on search_flight_details)
    state['flight_info'] = flight_info
    state['messages'].append(SystemMessage(content=f"Flight info updated: {flight_info}"))
    return state

In [65]:
def daily_schedule_node(state: State) -> State:
    trip_start = state.get("trip_start_date")
    trip_end = state.get("trip_end_date")
    preferences = state.get("traveler_preferences", {})
    flight_info = state.get("flight_info", {})
    city = state.get("city", "")  # Added to ensure city is included

    user_prompt = json.dumps(
        {
            "instructions": (
                "Generate a diverse activity pool for the specified city and travel dates. "
                "Include activities of type 'activity' (covering food, culture, and light hiking), 'meal', 'transit', and 'rest'. "
                "Food activities should include local restaurants, street food, or cooking classes. "
                "Culture activities should include museums, historical sites, or festivals. "
                "Light hiking should include scenic trails or parks suitable for light outdoor activity. "
                "Tailor activities to the city, travel dates (considering seasonal factors), and preferences (e.g., preferred activity types or time constraints). "
                "Schedule meals within lunch (12:00-13:00) and dinner (18:00-19:00) windows. "
                "Account for average transit time (20 minutes) between activities. "
                "If preferences are missing, include a balanced mix of food, culture, and light hiking. "
                "For multi-city trips, include transitions between cities based on flight_info. "
                "Return a JSON object matching the required_output_format with 'days', 'transitions', and 'segments'. "
                "Each day should have a balanced schedule from 09:00 to 20:00, respecting transit and meal times."
            ),
            "trip": {
                "city": city,
                "start_date": trip_start,
                "end_date": trip_end,
                "preferences": preferences,
            },
            "flight_info": flight_info,
            "assumptions": {
                "default_open": "09:00-20:00",
                "avg_transit_min": 20,
                "lunch_window": "12:00-13:00",
                "dinner_window": "18:00-19:00",
            },
            "required_output_format": {
                "days": [
                    {
                        "date": "YYYY-MM-DD",
                        "location": "City/Area",
                        "items": [
                            {
                                "type": "activity|meal|transit|rest",
                                "name": "string",
                                "notes": "string",
                                "start_min": 540,  # Minutes since midnight (e.g., 09:00)
                                "end_min": 600     # Minutes since midnight (e.g., 10:00)
                            }
                        ]
                    }
                ],
                "transitions": [
                    {"date": "YYYY-MM-DD", "from": "A", "to": "B", "notes": "string"}
                ],
                "segments": [
                    {
                        "location": "City/Area",
                        "start_date": "YYYY-MM-DD",
                        "end_date": "YYYY-MM-DD",
                        "days": 3
                    }
                ]
            }
        }
    )

    system_prompt = get_system_prompt(
        "Return ONLY a JSON object matching required_output_format. Do not include any prose."
    )

    messages = [SystemMessage(content=system_prompt), HumanMessage(content=user_prompt)]
    schedule_output = llm.invoke(messages)
    schedule_content = getattr(schedule_output, "content", None) or schedule_output

    try:
        activity_pool = json.loads(schedule_content)
        state["total_schedule"] = activity_pool
        return state
    except json.JSONDecodeError:
        state["total_schedule"] = {"days": [], "transitions": [], "segments": []}
        state["messages"].append(SystemMessage(content="Failed to parse activity pool JSON"))
        return state

In [66]:
def report_node(state: State) -> State:
    total = state.get("total_schedule", {"days": [], "transitions": [], "segments": []})
    flight_info = state.get("flight_info", {})
    lines = []
    lines.append(f"# Trip Plan: {state['trip_start_date']} to {state['trip_end_date']}")
    lines.append("")
    # Overview
    lines.append("## Overview")
    if total.get("segments"):
        lines.append("- Segments:")
        for seg in sorted(total.get("segments", []), key=lambda s: s.get("start_date", "")):
            lines.append(f"  - {seg.get('location','')}: {seg.get('start_date','')} → {seg.get('end_date','')} ({seg.get('days',0)} days)")
    if total.get("transitions"):
        lines.append("- Transitions:")
        for t in total.get("transitions", []):
            lines.append(f"  - {t.get('date','')}: {t.get('from','')} → {t.get('to','')} — {t.get('notes','')}")
    if flight_info:
        lines.append("- Flights:")
        start_f = flight_info.get("start", {})
        ret_f = flight_info.get("return", {})
        if start_f:
            lines.append(f"  - Outbound {start_f.get('flight_no','')}: {start_f.get('depart','')} → {start_f.get('arrive','')} ({start_f.get('date','')})")
        if ret_f:
            lines.append(f"  - Return {ret_f.get('flight_no','')}: {ret_f.get('depart','')} → {ret_f.get('arrive','')} ({ret_f.get('date','')})")
    lines.append("")

    # Daily tables
    lines.append("## Daily Schedule")
    for day in sorted(total.get("days", []), key=lambda d: d.get("date", "")):
        lines.append(f"### {day.get('date', '')} — {day.get('location', '')}")
        lines.append("| Time | Type | Name | Notes |")
        lines.append("|------|------|------|-------|")
        for item in day.get("items", []):
            start = item.get("start_min", 0)
            end = item.get("end_min", 0)
            def _hm(m):
                h = int(m // 60)
                mm = int(m % 60)
                return f"{h:02d}:{mm:02d}"
            time_str = f"{_hm(start)}–{_hm(end)}" if end else _hm(start)
            lines.append(f"| {time_str} | {item.get('type','activity')} | {item.get('name','')} | {item.get('notes','')} |")
        lines.append("")

    state["report_markdown"] = "\n".join(lines)
    return state

In [67]:
workflow = StateGraph(State)

# Nodes
workflow.add_node("flight_data", flight_data_node)
workflow.add_node("daily_schedule", daily_schedule_node)
workflow.add_node("report", report_node)

# Edges
workflow.add_edge(START, "flight_data")
workflow.add_edge("flight_data", "daily_schedule")
workflow.add_edge("daily_schedule", "report")
workflow.add_edge("report", END)

# Entry point
workflow.set_entry_point("flight_data")

graph = workflow.compile()

In [68]:
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	flight_data(flight_data)
	daily_schedule(daily_schedule)
	report(report)
	__end__([<p>__end__</p>]):::last
	__start__ --> flight_data;
	daily_schedule --> report;
	flight_data --> daily_schedule;
	report --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [69]:
graph.get_graph().print_ascii()

  +-----------+    
  | __start__ |    
  +-----------+    
         *         
         *         
         *         
  +-------------+  
  | flight_data |  
  +-------------+  
         *         
         *         
         *         
+----------------+ 
| daily_schedule | 
+----------------+ 
         *         
         *         
         *         
    +--------+     
    | report |     
    +--------+     
         *         
         *         
         *         
    +---------+    
    | __end__ |    
    +---------+    


### Run Graph

In [70]:
input_message = HumanMessage(
    content=(
        "Plan a 10-day Japan trip arriving Kansai (Osaka). Focus on food, culture, "
        "light hiking, and efficient transit."
    )
)

In [71]:
TRIP_START = "2025-03-10"
TRIP_END = "2025-03-24"
START_FLIGHT = "UO870"
RETURN_FLIGHT = "UO871"
PREFERENCES = {
    "pace": "moderate",
    "interests": ["food", "culture", "light hiking"],
    "budget": "mid",
}

#EMAIL_ADDRESS = "terence2379@gmail.com"

In [72]:
events = graph.stream(
    input={
        "messages": [input_message],
        "trip_start_date": TRIP_START,
        "trip_end_date": TRIP_END,
        "start_flight": START_FLIGHT,
        "return_flight": RETURN_FLIGHT,
        "traveler_preferences": PREFERENCES,
    },
    config={"recursion_limit": 50},  # Increased from 30
    stream_mode="values",
)
final_report = None
for event in events:
    if "report_markdown" in event:
        final_report = event["report_markdown"]

if final_report:
    display(Markdown(final_report))
else:
    print("No final report produced. Check inputs or graph configuration.")

# Trip Plan: 2025-03-10 to 2025-03-24

## Overview
- Flights:

## Daily Schedule